In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
# нужные библиотеки
import numpy as np
import pandas as pd
import random
import numpy as np

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from preprocessing import preprocess_df

import lightgbm as lgbm

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [4]:
THRESHOLD = 0.15
NEGATIVE_WEIGHT = 1.1

def deviation_metric_one_sample(y_true: typing.Union[float, int], y_pred: typing.Union[float, int]) -> float:
    """
    Реализация кастомной метрики для хакатона.

    :param y_true: float, реальная цена
    :param y_pred: float, предсказанная цена
    :return: float, значение метрики
    """
    deviation = (y_pred - y_true) / np.maximum(1e-8, y_true)
    if np.abs(deviation) <= THRESHOLD:
        return 0
    elif deviation <= - 4 * THRESHOLD:
        return 9 * NEGATIVE_WEIGHT
    elif deviation < -THRESHOLD:
        return NEGATIVE_WEIGHT * ((deviation / THRESHOLD) + 1) ** 2
    elif deviation < 4 * THRESHOLD:
        return ((deviation / THRESHOLD) - 1) ** 2
    else:
        return 9


def deviation_metric(y_true: np.array, y_pred: np.array) -> float:
    return np.array([deviation_metric_one_sample(y_true[n], y_pred[n]) for n in range(len(y_true))]).mean()


In [20]:
# загрузка данных
train = pd.read_csv('../../../Данные хакатона Raifhack DS/data/train.csv')
test = pd.read_csv('../../../Данные хакатона Raifhack DS/data/test.csv')

test_submission = pd.read_csv('../../../Данные хакатона Raifhack DS/data/test_submission.csv', low_memory=False)

/Users/al_kharba/opt/anaconda3/envs/cv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
train, target = preprocess_df(train)
test, _= preprocess_df(test, use_target=False)

In [19]:
# подбор параметров для LightGBM
import optuna
from optuna.samplers import TPESampler
sampler = TPESampler(seed=13)

def create_model(trial):
    num_leaves = trial.suggest_int("num_leaves", 2, 1500)
    n_estimators = trial.suggest_int("n_estimators", 10, 2000)
    max_depth = trial.suggest_int('max_depth', 2, 25)
    min_child_samples = trial.suggest_int('min_child_samples', 2, 300)
    learning_rate = trial.suggest_uniform('learning_rate', 0.00001, 0.99)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 2, 300)
    feature_fraction = trial.suggest_uniform('feature_fraction', 0.00001, 1.0)
    
    model = lgbm.LGBMRegressor(
        num_leaves=num_leaves,
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        min_child_samples=min_child_samples, 
        min_data_in_leaf=min_data_in_leaf,
        learning_rate=learning_rate,
        feature_fraction=feature_fraction,
        random_state=13,
        n_jobs=-1,
        objective='regression_l1'
)
    return model

def objective(trial):
    model = create_model(trial)
    X_train, X_test, y_train, y_test = train_test_split(train, target, random_state=random.randint(1, 10000))
    model.fit(X_train, y_train)
    result = model.predict(X_test)
    score = deviation_metric(np.exp(y_test), np.exp(result))
    return score

In [20]:
study = optuna.create_study(direction="minimize", sampler=sampler)
study.optimize(objective, n_trials=100)
params_lgbm = study.best_params

[I 2021-09-25 21:49:15,031] A new study created in memory with name: no-name-0318f32f-1e2e-4934-98b9-d819afcb8107
[I 2021-09-25 21:49:15,722] Trial 0 finished with value: 1.8648288530003048 and parameters: {'num_leaves': 1167, 'n_estimators': 482, 'max_depth': 21, 'min_child_samples': 290, 'learning_rate': 0.9628753767547054, 'min_data_in_leaf': 137, 'feature_fraction': 0.6090463723366503}. Best is trial 0 with value: 1.8648288530003048.


[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=12 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] feature_fraction is set=0.857062371977773, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.857062371977773


[I 2021-09-25 21:49:23,809] Trial 1 finished with value: 1.279002387915969 and parameters: {'num_leaves': 1164, 'n_estimators': 1287, 'max_depth': 19, 'min_child_samples': 12, 'learning_rate': 0.29547199168557875, 'min_data_in_leaf': 19, 'feature_fraction': 0.857062371977773}. Best is trial 1 with value: 1.279002387915969.


[LightGBM] [Warning] min_data_in_leaf is set=109, min_child_samples=105 will be ignored. Current value: min_data_in_leaf=109
[LightGBM] [Warning] feature_fraction is set=0.9490946907363935, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9490946907363935


[I 2021-09-25 21:49:25,570] Trial 2 finished with value: 1.5856707639814585 and parameters: {'num_leaves': 560, 'n_estimators': 1363, 'max_depth': 8, 'min_child_samples': 105, 'learning_rate': 0.009328548252458909, 'min_data_in_leaf': 109, 'feature_fraction': 0.9490946907363935}. Best is trial 1 with value: 1.279002387915969.


...

[I 2021-09-25 21:58:32,976] Trial 97 finished with value: 1.4019915585431637 and parameters: {'num_leaves': 1480, 'n_estimators': 1693, 'max_depth': 18, 'min_child_samples': 163, 'learning_rate': 0.16839421570301558, 'min_data_in_leaf': 42, 'feature_fraction': 0.7095752631997853}. Best is trial 43 with value: 1.2255143151507202.


[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=182 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] feature_fraction is set=0.8061499465737154, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8061499465737154


[I 2021-09-25 21:58:41,740] Trial 98 finished with value: 1.4332307805586382 and parameters: {'num_leaves': 871, 'n_estimators': 1784, 'max_depth': 12, 'min_child_samples': 182, 'learning_rate': 0.11744610415560885, 'min_data_in_leaf': 14, 'feature_fraction': 0.8061499465737154}. Best is trial 43 with value: 1.2255143151507202.


[LightGBM] [Warning] min_data_in_leaf is set=49, min_child_samples=170 will be ignored. Current value: min_data_in_leaf=49
[LightGBM] [Warning] feature_fraction is set=0.6817290030902752, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6817290030902752


[I 2021-09-25 21:58:46,672] Trial 99 finished with value: 1.3513143805058778 and parameters: {'num_leaves': 1465, 'n_estimators': 1923, 'max_depth': 13, 'min_child_samples': 170, 'learning_rate': 0.2374004905299345, 'min_data_in_leaf': 49, 'feature_fraction': 0.6817290030902752}. Best is trial 43 with value: 1.2255143151507202.


In [21]:
params_lgbm = study.best_params
params_lgbm['objective'] = 'regression_l1'
params_lgbm

{'num_leaves': 1149,
 'n_estimators': 1161,
 'max_depth': 15,
 'min_child_samples': 50,
 'learning_rate': 0.04185199859407067,
 'min_data_in_leaf': 14,
 'feature_fraction': 0.6494305066563237,
 'objective': 'regression_l1'}

In [22]:
# заведем модель
model = lgbm.LGBMRegressor(**params_lgbm)

In [24]:
%%time

# параметры валидации, обучение будет идти на 5х10 фолдах
kFold_random_state = [13, 666, 228, 777, 42]
n_splits = 10

# финальный показатель метрики
final_loss = list()

# файл для записи финального результата
submission = test_submission.copy()
submission.iloc[:, 1] = 0


for ind_k, random_state in enumerate(kFold_random_state):
    kFold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    total_loss = list()

    for iteration, (train_index, valid_index) in enumerate(kFold.split(train, target)):

        X_train, X_valid = train.iloc[train_index].reset_index(drop=True), train.iloc[valid_index].reset_index(drop=True)
        y_train, y_valid = target[train_index], target[valid_index]

        model.fit(X_train, y_train)
        valid_pred = model.predict(X_valid)
        loss = deviation_metric(np.exp(y_valid), np.exp(valid_pred))

        predict = model.predict(test)
        submission['per_square_meter_price'] = submission['per_square_meter_price'] + np.exp(predict) / 50

        total_loss.append(np.mean(loss))

    final_loss.append(np.mean(total_loss))
    print(f'Fold({["1-10", "11-20", "21-30", "31-40", "41-50"][ind_k]}) deviation_metric: {np.mean(total_loss)}')
print(f'Final deviation_metric: {np.mean(final_loss)}')


[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=50 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] feature_fraction is set=0.6494305066563237, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6494305066563237
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=50 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] feature_fraction is set=0.6494305066563237, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6494305066563237
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=50 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] feature_fraction is set=0.6494305066563237, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6494305066563237
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=50 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] feature_fraction is set=0.649

In [29]:
np.mean(final_loss)

1.24388647997935

In [30]:
submission_raw = submission.copy()

In [31]:
# немного подрежем выбросы сверху
submission_raw['per_square_meter_price'] = submission_raw['per_square_meter_price'] * 0.9

submission_raw.loc[submission_raw['per_square_meter_price'] >= 200000, 'per_square_meter_price'] \
    = submission_raw.loc[submission_raw['per_square_meter_price'] >= 200000, 'per_square_meter_price'] * 0.9

In [32]:
submission_raw.to_csv('final_submission_log_target_and_feat_imp.csv', index=False)

In [ ]:
#features_imp = pd.DataFrame({
#    'name' : train.columns,
#    'imp' : model.feature_importances_
#})
#features_imp.to_csv('9th_place_sol_feat_imp.csv', index=False)